# 0.0 Imports

In [22]:
!pip install inflection

In [23]:
!pip install xgboost==0.90

In [24]:
import math
import pickle

import pandas as pd
import numpy as np
import inflection

import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb

from scipy import stats as ss
#from boruta import BorutaPy

from sklearn.preprocessing import RobustScaler, MinMaxScaler, LabelEncoder
from sklearn.ensemble      import RandomForestRegressor
from sklearn.linear_model  import LinearRegression, Lasso
from sklearn.metrics       import mean_absolute_error, mean_squared_error #mean_absolute_percentage_error

from IPython.core.display import HTML
from IPython.display      import Image

## 0.1 Helper Functions

In [25]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    
    sns.set()
    
def cramer_v( x, y ):
    cm = pd.crosstab( x, y ).values
    n = cm.sum()
    r, k = cm.shape

    chi2 = ss.chi2_contingency( cm )[0]
    chi2corr = max( 0, chi2 - (k-1)*(r-1)/(n-1) )

    kcorr = k - (k-1)**2/(n-1)
    rcorr = r - (r-1)**2/(n-1)
    return np.sqrt( (chi2corr/n) / ( min( kcorr-1, rcorr-1 ) ) )

def mean_percentage_error( y, yhat ):
    return np.mean( ( y - yhat ) / y )
     
    
def mean_absolute_percentage_error( y, yhat ):
    return np.mean( np.abs( ( y - yhat ) / y ) )

    
def ml_error( model_name, y, yhat ):
    mae = mean_absolute_error( y, yhat )
    mape = mean_absolute_percentage_error( y, yhat )
    rmse = np.sqrt( mean_squared_error( y, yhat ) )
    
    return pd.DataFrame( { 'Model Name': model_name, 
                           'MAE': mae, 
                           'MAPE': mape,
                           'RMSE': rmse }, index=[0] )

def ts_cross_validation( x_training, kfold, model_name, model, verbose=False ):
    mae_list = []
    mape_list = []
    rmse_list = []
    for k in reversed( range( 1, kfold+1 ) ):
        if verbose:
            print( '\nKFold Number: {}'.format( k ) )
        # start and end date for validation 
        validation_start_date = x_training['date'].max() - datetime.timedelta( days=k*6*7)
        validation_end_date = x_training['date'].max() - datetime.timedelta( days=(k-1)*6*7)

        # filtering dataset
        training = x_training[x_training['date'] < validation_start_date]
        validation = x_training[(x_training['date'] >= validation_start_date) & (x_training['date'] <= validation_end_date)]

        # training and validation dataset
        # training
        xtraining = training.drop( ['date', 'sales'], axis=1 ) 
        ytraining = training['sales']

        # validation
        xvalidation = validation.drop( ['date', 'sales'], axis=1 )
        yvalidation = validation['sales']

        # model
        m = model.fit( xtraining, ytraining )

        # prediction
        yhat = m.predict( xvalidation )

        # performance
        m_result = ml_error( model_name, np.expm1( yvalidation ), np.expm1( yhat ) )

        # store performance of each kfold iteration
        mae_list.append(  m_result['MAE'] )
        mape_list.append( m_result['MAPE'] )
        rmse_list.append( m_result['RMSE'] )

    return pd.DataFrame( {'Model Name': model_name,
                          'MAE CV': np.round( np.mean( mae_list ), 2 ).astype( str ) + ' +/- ' + np.round( np.std( mae_list ), 2 ).astype( str ),
                          'MAPE CV': np.round( np.mean( mape_list ), 2 ).astype( str ) + ' +/- ' + np.round( np.std( mape_list ), 2 ).astype( str ),
                          'RMSE CV': np.round( np.mean( rmse_list ), 2 ).astype( str ) + ' +/- ' + np.round( np.std( rmse_list ), 2 ).astype( str ) }, index=[0] )


In [26]:
jupyter_settings()

## 0.2 Loading Data

In [27]:
df_sales_raw = pd.read_csv("/kaggle/input/rossmann-store-sales/train.csv", low_memory=False)
df_store_raw = pd.read_csv("/kaggle/input/rossmann-store-sales/store.csv", low_memory=False)

In [28]:
df_sales_raw.head()

In [29]:
df_store_raw.head()

In [30]:
df_raw = pd.merge(df_sales_raw, df_store_raw, how='left', on='Store')

## 0.3 Data Dictionary

* **Id** - an Id that represents a (Store, Date) duple within the test set

* **Store** - a unique Id for each store

* **Sales** - the turnover for any given day (this is what you are predicting)

* **Customers** - the number of customers on a given day

* **Open** - an indicator for whether the store was open: 0 = closed, 1 = open

* **StateHoliday** - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b  Easter holiday, c = Christmas, 0 = None

* **SchoolHoliday** - indicates if the (Store, Date) was affected by the closure of public schools

* **StoreType** - differentiates between 4 different store models: a, b, c, d

* **Assortment** - describes an assortment level: a = basic, b = extra, c = extended

* **CompetitionDistance** - distance in meters to the nearest competitor store

* **CompetitionOpenSince[Month/Year]** - gives the approximate year and month of the time the nearest competitor was opened

* **Promo** - indicates whether a store is running a promo on that day

* **Promo2** - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating

* **Promo2Since[Year/Week]** - describes the year and calendar week when the store started participating in Promo2

* **PromoInterval** - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store

# 1.0 Data Description

## 1.1 Rename columns

In [31]:
df1 = df_raw.copy()

In [32]:
df1.head()

In [33]:
df1.columns

In [34]:
oldcols = ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval']

snakecase = lambda x: inflection.underscore(x)

cols_news = list(map(snakecase, oldcols))

df1.columns = cols_news

## 1.2 Data Exploration

In [35]:
#Data dimensions

print("number of rows: {}".format(len(df1)))
print("number of columns: {}".format(df1.shape[1]))

In [36]:
#Data types

df1.info()

## 1.3 Dates to datetime

In [37]:
df1['date'] = pd.to_datetime(df1['date'])

In [38]:
df1.info()

## 1.4 Check and fill NA

In [39]:
df1.isnull().sum()

In [40]:
print(df1['competition_distance'].max())
print(df1['competition_open_since_year'].max())

In [41]:
#competition_distance
df1['competition_distance'] = df1['competition_distance'].apply(lambda x: 200000 if math.isnan(x) else x)

#competition_open_since_month    
df1['competition_open_since_month'] = df1.apply(lambda x: x['date'].month if math.isnan(x['competition_open_since_month']) else x['competition_open_since_month'], axis=1)

#competition_open_since_year     
df1['competition_open_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['competition_open_since_year']) else x['competition_open_since_year'], axis=1)

#promo2_since_week         
df1['promo2_since_week'] = df1.apply(lambda x: x['date'].week if math.isnan(x['promo2_since_week']) else x['promo2_since_week'], axis=1)

#promo2_since_year
df1['promo2_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['promo2_since_year']) else x['promo2_since_year'], axis=1)

#promo_interval                  

month_map = {1: 'Jan', 2: 'Fev', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7: 'Jul', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'}

df1['promo_interval'].fillna(0, inplace=True)
df1['month_map'] = df1['date'].dt.month.map(month_map)
df1['is_promo'] = df1[['promo_interval', 'month_map']].apply(lambda x: 0 if x['promo_interval'] == 0 else 1 if x['month_map'] in x['promo_interval'].split(',') else 0, axis=1)

In [42]:
df1.isnull().sum()

## 1.5 Change dtypes

In [43]:
df1.info()

In [44]:
df1['competition_open_since_month'] = df1['competition_open_since_month'].astype(int)
df1['competition_open_since_year'] = df1['competition_open_since_year'].astype(int)
df1['promo2_since_week'] = df1['promo2_since_week'].astype(int)
df1['promo2_since_year'] = df1['promo2_since_year'].astype(int)

In [45]:
df1.info()

## 1.6 Descriptive Statistics

In [46]:
num_attributes = df1.select_dtypes(include=['int64', 'float64'])
cat_attributes = df1.select_dtypes(exclude=['int64', 'float64', 'datetime64[ns]'])

### 1.6.1 Numerical Attributes

In [47]:
#Central tendency - mean, median
ct1 = pd.DataFrame(num_attributes.apply(np.mean)).transpose()
ct2 = pd.DataFrame(num_attributes.apply(np.median)).transpose()

#Dispersion - std, min, max, range, skew, kurtosis
d1 = pd.DataFrame(num_attributes.apply(np.std)).transpose()
d2 = pd.DataFrame(num_attributes.apply(np.min)).transpose()
d3 = pd.DataFrame(num_attributes.apply(np.max)).transpose()
d4 = pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min())).transpose()
d5 = pd.DataFrame(num_attributes.apply(lambda x: x.skew())).transpose()
d6 = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis())).transpose()

#concatenate
m = pd.concat([ct1, ct2, d1, d2, d3, d4, d5, d6]).transpose().reset_index()
m.columns = ['index', 'mean', 'median', 'std', 'min', 'max', 'range', 'skew', 'kurtosis']

In [48]:
m

In [49]:
sns.distplot(x=df1['sales'])

### 1.6.2 Categorical attributes

In [50]:
cat_attributes.apply(lambda x: x.unique().shape[0])

In [51]:
aux1 = df1[(df1['state_holiday'] != '0') & (df1['sales'] > 0)]

plt.subplot(1, 3, 1)
sns.boxplot(x='state_holiday', y='sales', data=aux1)

plt.subplot(1, 3, 2)
sns.boxplot(x='store_type', y='sales', data=aux1)

plt.subplot(1, 3, 3)
sns.boxplot(x='assortment', y='sales', data=aux1)

# 2.0 Feature Engineering

In [52]:
df2 = df1.copy()

In [53]:
#Image("/home/mvrcosp/repos/DSP/Rossmann/img/EngMindMapHypothesis.png")

## 2.1 Brainstorming Business Hypothesis to validate with data!

### 2.1.1 Store Hypothesis

**1.** Stores with a bigger number of employees should sell more.

**2.** Stores with a bigger stock size should sell more.

**3.** Stores with a bigger size should sell more.

**4.** Stores with local competitors should sell less.

**5.** Stores with longer-term competitors should sell more.

**6.** Stores with bigger assortment should sell more.

### 2.1.2 Product Hypothesis

**1.** Stores that invest in marketing strategies should sell more.

**2.** Stores that showcase their product better should sell more.

**3.** Stores with cheaper products should sell more.

**4.** Stores that perform more agressive promos should sell more.

**5.** Stores that keep their promos active for longer periods should sell more.

**6** Stores with consecutive promos should sell more.

### 2.1.3 Sazonality Hypothesis

**1.** Stores that open during christmas season should sell more.

**2.** Stores should sell more over the years.

**3.** Stores should sell more at the 2nd semester of the year.

**4.** Stores should sell more at the beginning of each month.

**5.** Stores should sell more on weekends.

**6.** Stores should sell less on school holidays.

## 2.2 Final Business Hypothesis List

**H1.** Stores with bigger assortment should sell more.

**H2.** Stores with local competitors should sell less.

**H3.** Stores with longer-term competitors should sell more.

**H4.** Stores that keep their promos active for longer periods should sell more.

**H5.** Stores with consecutive promos should sell more.

**H6.** Stores that open during christmas season should sell more.

**H7.** Stores should sell more over the years.

**H8.** Stores should sell more at the 2nd semester of the year.

**H9.** Stores should sell more at the beginning of each month.

**H10.** Stores should sell less on weekends.

**H11.** Stores should sell less on school holidays.

## 2.3 Feature Engineering

In [54]:
df2['date'] = pd.to_datetime(df2['date'])

# year
df2['year'] = df2['date'].dt.year

# month
df2['month'] = df2['date'].dt.month

# day
df2['day'] = df2['date'].dt.day

# Week of year
df2['week_of_year'] = df2['date'].dt.weekofyear

#year week
df2['year_week'] = df2['date'].dt.strftime('%Y-%W')

#competition since
df2['competition_since'] = df2.apply( lambda x: datetime.datetime( year=x['competition_open_since_year'], month=x['competition_open_since_month'],day=1 ), axis=1 )
df2['competition_time_month'] = ( ( df2['date'] - df2['competition_since'] )/30 ).apply( lambda x: x.days ).astype( int )

# promo since
df2['promo_since'] = df2['promo2_since_year'].astype( str ) + '-' + df2['promo2_since_week'].astype( str )
df2['promo_since'] = df2['promo_since'].apply( lambda x: datetime.datetime.strptime( x + '-1', '%Y-%W-%w' ) - datetime.timedelta( days=7 ) )
df2['promo_time_week'] = ( ( df2['date'] - df2['promo_since'] )/7 ).apply( lambda x: x.days ).astype( int )

# assortment
df2['assortment'] = df2['assortment'].apply( lambda x: 'basic' if x == 'a' else 'extra' if x == 'b' else 'extended' )

# state holiday
df2['state_holiday'] = df2['state_holiday'].apply( lambda x: 'public_holiday' if x == 'a' else 'easter_holiday' if x == 'b' else 'christmas' if x == 'c' else 'regular_day' )

# 3.0 Data Filtering

In [55]:
df3 = df2.copy()

In [56]:
df3 = df3[(df3["open"] != 0) & (df3['sales'] > 0)]

In [57]:
cols_drop = ['customers', 'open', 'promo_interval', 'month_map']
df3.drop(cols_drop, inplace=True, axis=1)

In [58]:
df3.head()

In [59]:
df3.dtypes

# 4.0 Exploratory Data Analysis

In [60]:
df4 = df3.copy()

## 4.1 Univariate analysis

### 4.1.1 Target Variable

In [ ]:
sns.distplot(df4['sales'], kde=False)

### 4.1.2 Numerical Variables

In [ ]:
num_attributes.hist(bins=25)
plt.show()

### 4.1.3 Categorical Variables

In [ ]:
# state_holiday
plt.subplot( 3, 2, 1 )
a = df4[df4['state_holiday'] != 'regular_day']
sns.countplot( x = 'state_holiday', data = a )

plt.subplot( 3, 2, 2 )
sns.kdeplot( df4[df4['state_holiday'] == 'public_holiday']['sales'], label='public_holiday', shade=True )
sns.kdeplot( df4[df4['state_holiday'] == 'easter_holiday']['sales'], label='easter_holiday', shade=True )
sns.kdeplot( df4[df4['state_holiday'] == 'christmas']['sales'], label='christmas', shade=True )

# store_type
plt.subplot( 3, 2, 3 )
sns.countplot(x = 'store_type', data = df4)

plt.subplot( 3, 2, 4 )
sns.kdeplot( df4[df4['store_type'] == 'a']['sales'], label='a', shade=True )
sns.kdeplot( df4[df4['store_type'] == 'b']['sales'], label='b', shade=True )
sns.kdeplot( df4[df4['store_type'] == 'c']['sales'], label='c', shade=True )
sns.kdeplot( df4[df4['store_type'] == 'd']['sales'], label='d', shade=True )

# assortment
plt.subplot( 3, 2, 5 )
sns.countplot( x = 'assortment', data = df4)

plt.subplot( 3, 2, 6 )
sns.kdeplot( df4[df4['assortment'] == 'extended']['sales'], label='extended', shade=True )
sns.kdeplot( df4[df4['assortment'] == 'basic']['sales'], label='basic', shade=True )
sns.kdeplot( df4[df4['assortment'] == 'extra']['sales'], label='extra', shade=True )

plt.show()

## 4.2 Bivariate Analysis - Validating Business Hypothesis

### 4.2.1 - H1 - Stores with bigger assortment should sell more.

**False:** Stores with extended assortment do sell more than stores with basic assortment, but, stores that sell the most are the ones with only "extra" assortment (mot extended).

In [ ]:
aux421_1 = df4[['assortment', 'sales']].groupby('assortment').mean().reset_index()
sns.barplot(x = 'assortment', y ='sales', data = aux421_1)

In [ ]:
aux421_2 = df4[['year_week', 'assortment', 'sales']].groupby(['year_week', 'assortment']).mean().reset_index()
aux421_2.pivot(index='year_week', columns='assortment', values='sales').plot()

In [ ]:
aux421_2[aux421_2['assortment'] =='extra'].plot()

### 4.2.2 - H2 - Stores with local competitors should sell less.
**False:** Stores with competitors close by, in reality, sell more.

In [ ]:
aux422_1 = df4[['competition_distance', 'sales']].groupby('competition_distance').sum().reset_index()

plt.subplot(1, 3, 1)
sns.scatterplot(x='competition_distance', y='sales', data=aux422_1)


plt.subplot(1, 3, 2)
bins = list(np.arange(0, 20000, 1000))
aux422_1['competition_distance_binned'] = pd.cut(aux422_1['competition_distance'], bins = bins)
aux422_2 = aux422_1[['competition_distance_binned', 'sales']].groupby('competition_distance_binned').sum().reset_index()
sns.barplot(x='competition_distance_binned', y='sales', data=aux422_2)
plt.xticks(rotation=45)

plt.subplot(1, 3, 3)
heat = sns.heatmap(aux422_1.corr(method='pearson'), annot=True)

plt.show()

### 4.2.3 - H3 - Stores with longer-term competitors should sell more.
**False:** In reality, stores with longer-term competitiors sell less.

In [ ]:
aux423_1 = df4[['competition_time_month', 'sales']].groupby( 'competition_time_month' ).sum().reset_index()
aux423_2 = aux423_1[( aux423_1['competition_time_month'] < 120 ) & ( aux423_1['competition_time_month'] != 0 )]
sns.barplot( x='competition_time_month', y='sales', data=aux423_2 );
plt.xticks( rotation=90 );

In [ ]:
plt.subplot( 1, 2, 1 )
sns.scatterplot( x='competition_time_month', y='sales', data=aux423_2 );

plt.subplot( 1, 2, 2 )
x = sns.heatmap( aux423_1.corr( method='pearson'), annot=True );

### 4.2.4 - H4 - Stores that keep their promos active for longer periods should sell more.

**False:** Actually sales start to drop after a few weeks in promotion.

In [ ]:
aux424_1 = df4[['promo_time_week', 'sales']].groupby( 'promo_time_week').sum().reset_index()

grid = GridSpec( 2, 3 )

plt.subplot( grid[0,0] )
aux424_2 = aux424_1[aux424_1['promo_time_week'] > 0] # promo extendido
sns.barplot( x='promo_time_week', y='sales', data=aux424_2 );
plt.xticks( rotation=90 );

plt.subplot( grid[0,1] )
sns.regplot( x='promo_time_week', y='sales', data=aux424_2 );

plt.subplot( grid[1,0] )
aux424_3 = aux424_1[aux424_1['promo_time_week'] < 0] # promo regular
sns.barplot( x='promo_time_week', y='sales', data=aux424_3 );
plt.xticks( rotation=90 );

plt.subplot( grid[1,1] )
sns.regplot( x='promo_time_week', y='sales', data=aux424_3 );

plt.subplot( grid[:,2] )
sns.heatmap( aux424_1.corr( method='pearson' ), annot=True );

### 4.2.5 - H5 - Stores with consecutive promos should sell more.

**False:** Stores with consecutive promos actually sell less than stores with only promo 1.

In [ ]:
df4[['promo', 'promo2', 'sales']].groupby( ['promo', 'promo2'] ).sum().sort_values(by='sales').reset_index()

In [ ]:
 aux425_1 = df4[( df4['promo'] == 1 ) & ( df4['promo2'] == 1 )][['year_week', 'sales']].groupby( 'year_week' ).sum().reset_index()
ax = aux425_1.plot()

aux425_2 = df4[( df4['promo'] == 1 ) & ( df4['promo2'] == 0 )][['year_week', 'sales']].groupby( 'year_week' ).sum().reset_index()
aux425_2.plot( ax=ax )

ax.legend( labels=['Promo 1 & Promo 2', 'Promo 1']);

### 4.2.6 - H6 - Stores that open during christmas season should sell more.

**False:** Public holidays sell better than christmas. Sorry santa :(

In [ ]:
aux426_1 = df4[df4['state_holiday'] != 'regular_day']

plt.subplot( 1, 2, 1 )
aux426_2 = aux426_1[['state_holiday', 'sales']].groupby( 'state_holiday' ).sum().reset_index()
sns.barplot( x='state_holiday', y='sales', data=aux426_2 );

plt.subplot( 1, 2, 2 )
aux426_3 = aux426_1[['year', 'state_holiday', 'sales']].groupby( ['year', 'state_holiday'] ).sum().reset_index()
sns.barplot( x='year', y='sales', hue='state_holiday', data=aux426_3 ); 

### 4.2.7 - H7 - Stores should sell more over the years.

**False:** Stores are selling less over the years.

In [ ]:
 aux427_1 = df4[['year', 'sales']].groupby( 'year' ).sum().reset_index()

plt.subplot( 1, 3, 1 )
sns.barplot( x='year', y='sales', data=aux427_1 );

plt.subplot( 1, 3, 2 )
sns.regplot( x='year', y='sales', data=aux427_1 );

plt.subplot( 1, 3, 3 )
sns.heatmap( aux427_1.corr( method='pearson' ), annot=True );

### 4.2.8 - H8 - Stores should sell more at the 2nd semester of the year.

**False:** Stores sell more at first semester of the year.

In [ ]:
aux428_1 = df4[['month', 'sales']].groupby( 'month' ).sum().reset_index()

plt.subplot( 1, 3, 1 )
sns.barplot( x='month', y='sales', data=aux428_1 );

plt.subplot( 1, 3, 2 )
sns.regplot( x='month', y='sales', data=aux428_1 );

plt.subplot( 1, 3, 3 )
sns.heatmap( aux428_1.corr( method='pearson' ), annot=True );

### 4.2.9 - H9 - Stores should sell more at the beginning of each month.

**True:** Stores do sell a little bit more at the beginning of the month.

In [ ]:
plt.subplot(1, 2, 1)
aux429_1 = df4[['year', 'month', 'day', 'sales']].groupby(['year', 'month', 'day']).sum().reset_index()
sns.barplot(x='day', y='sales', data=aux429_1)

plt.subplot(1, 2, 2)
bins = list(np.arange(0, 40, 10))
aux429_1['days_binned'] = pd.cut(aux429_1['day'], bins = bins)
sns.barplot(x='days_binned', y='sales', data=aux429_1)

### 4.2.10 - H10 - Stores should sell less on weekends.

**True:** Stores DO sell less on weekends compared to weekdays.

In [ ]:
aux4210_1 = df4[['day_of_week', 'sales']].groupby( 'day_of_week' ).sum().reset_index()

plt.subplot( 1, 3, 1 )
sns.barplot( x='day_of_week', y='sales', data=aux4210_1 );

plt.subplot( 1, 3, 2 )
sns.regplot( x='day_of_week', y='sales', data=aux4210_1 );

plt.subplot( 1, 3, 3 )
sns.heatmap( aux4210_1.corr( method='pearson' ), annot=True );

### 4.2.11 - H11 - Stores should sell less on school holidays.

**True:** Stores do sell less on school holidays. August is the only month where school holidays actually sell more.

In [ ]:
aux4211_1 = df4[['school_holiday', 'sales']].groupby( 'school_holiday' ).sum().reset_index()
plt.subplot( 2, 1, 1 )
sns.barplot( x='school_holiday', y='sales', data=aux4211_1 );

aux4211_2 = df4[['month', 'school_holiday', 'sales']].groupby( ['month','school_holiday'] ).sum().reset_index()
plt.subplot( 2, 1, 2 )
sns.barplot( x='month', y='sales', hue='school_holiday', data=aux4211_2 );

### 4.2.12 - Final Hypothesis table

In [ ]:
from tabulate import tabulate

In [ ]:
tab =[['Hypothesis', 'Conclusion', 'Relevance'],
      ['H1', 'False', 'Low'],  
      ['H2', 'False', 'Medium'],  
      ['H3', 'False', 'Medium'],
      ['H4', 'False', 'Low'],
      ['H5', 'False', 'Low'],
      ['H6', 'False', 'Medium'],
      ['H7', 'False', 'High'],
      ['H8', 'True', 'High'],
      ['H9', 'True', 'High'],
      ['H10', 'True', 'High'],
      ['H11', 'True', 'Low']]  

print(tabulate(tab, headers='firstrow'))

## 4.3 Multivariate Analysis

### 4.3.1 Numerical Attributes

In [ ]:
correlation = num_attributes.corr(method='pearson')
sns.heatmap(correlation, annot=True)

### 4.3.2 Categorial Attributes

In [ ]:
b = df4.select_dtypes('object')
b.drop('year_week', axis=1, inplace=True)

In [ ]:
#Calculte Cramer V

b1 = cramer_v( b['state_holiday'], b['state_holiday'] )
b2 = cramer_v( b['state_holiday'], b['store_type'] )
b3 = cramer_v( b['state_holiday'], b['assortment'] )

b4 = cramer_v( b['store_type'], b['state_holiday'] )
b5 = cramer_v( b['store_type'], b['store_type'] )
b6 = cramer_v( b['store_type'], b['assortment'] )

b7 = cramer_v( b['assortment'], b['state_holiday'] )
b8 = cramer_v( b['assortment'], b['store_type'] )
b9 = cramer_v( b['assortment'], b['assortment'] )

# Final dataset
d = pd.DataFrame( {'state_holiday': [b1, b2, b3], 
               'store_type': [b4, b5, b6],
               'assortment': [b7, b8, b9]  })
d = d.set_index( d.columns )

sns.heatmap( d, annot=True )

# 5.0 Data Preparation

In [61]:
df5 = df4.copy()

## 5.1 Rescaling (Numerical Attributes)

In [62]:
a = df5.select_dtypes(include=['int64', 'float64'])
a.head()

In [63]:
 rs = RobustScaler()
mms = MinMaxScaler()

# competition distance
df5['competition_distance'] = rs.fit_transform( df5[['competition_distance']].values )
#pickle.dump( rs, open( '/home/mvrcosp/repos/DSP/Rossmann/Pickles/competition_distance_scaler.pkl', 'wb') )

# competition time month
df5['competition_time_month'] = rs.fit_transform( df5[['competition_time_month']].values )
#pickle.dump( rs, open( '/home/mvrcosp/repos/DSP/Rossmann/Pickles/competition_time_month_scaler.pkl', 'wb') )

# promo time week
df5['promo_time_week'] = mms.fit_transform( df5[['promo_time_week']].values )
#pickle.dump( rs, open( '/home/mvrcosp/repos/DSP/Rossmann/Pickles/promo_time_week_scaler.pkl', 'wb') )

# year
df5['year'] = mms.fit_transform( df5[['year']].values )
#pickle.dump( mms, open( '/home/mvrcosp/repos/DSP/Rossmann/Pickles/year_scaler.pkl', 'wb') )

## 5.2 Encoding (Categorical Attributes)

In [64]:
df5.head()

In [65]:
 # state_holiday - One Hot Encoding
df5 = pd.get_dummies(df5, prefix=['state_holiday'], columns=['state_holiday'])

# store_type - Label Encoding
le = LabelEncoder()
df5['store_type'] = le.fit_transform( df5['store_type'] )
#pickle.dump( le, open( '/home/mvrcosp/repos/DSP/Rossmann/Pickles/store_type_scaler.pkl', 'wb') )

# assortment - Ordinal Encoding
assortment_dict = {'basic': 1,  'extra': 2, 'extended': 3}
df5['assortment'] = df5['assortment'].map( assortment_dict )

## 5.3 Target Variable Transformation (Logarithm Transformation)

In [66]:
df5['sales'] = np.log1p( df5['sales'] )

In [67]:
sns.distplot(df5.sales)

## 5.4 Dealing With the Cyclic Nature of Time

In [68]:
# day of week
df5['day_of_week_sin'] = df5['day_of_week'].apply( lambda x: np.sin( x * ( 2. * np.pi/7 ) ) )
df5['day_of_week_cos'] = df5['day_of_week'].apply( lambda x: np.cos( x * ( 2. * np.pi/7 ) ) )

# month
df5['month_sin'] = df5['month'].apply( lambda x: np.sin( x * ( 2. * np.pi/12 ) ) )
df5['month_cos'] = df5['month'].apply( lambda x: np.cos( x * ( 2. * np.pi/12 ) ) )

# day 
df5['day_sin'] = df5['day'].apply( lambda x: np.sin( x * ( 2. * np.pi/30 ) ) )
df5['day_cos'] = df5['day'].apply( lambda x: np.cos( x * ( 2. * np.pi/30 ) ) )

# week of year
df5['week_of_year_sin'] = df5['week_of_year'].apply( lambda x: np.sin( x * ( 2. * np.pi/52 ) ) )
df5['week_of_year_cos'] = df5['week_of_year'].apply( lambda x: np.cos( x * ( 2. * np.pi/52 ) ) )

# 6.0 Feature Selection

In [69]:
df6 = df5.copy()

## 6.1 Split DataFrame into Traning and Test Dataset

In [70]:
cols_to_drop = ['week_of_year', 'day', 'month', 'day_of_week', 'promo_since', 'competition_since', 'year_week']
df6.drop(cols_to_drop, axis=1, inplace=True)

In [71]:
# training dataset
X_train = df6[df6['date'] < '2015-06-19']
y_train = X_train['sales']

# test dataset
X_test = df6[df6['date'] >= '2015-06-19']
y_test = X_test['sales']

print( 'Training Min Date: {}'.format( X_train['date'].min() ) )
print( 'Training Max Date: {}'.format( X_train['date'].max() ) )

print( '\nTest Min Date: {}'.format( X_test['date'].min() ) )
print( 'Test Max Date: {}'.format( X_test['date'].max() ) )

## 6.2 Boruta Algorithm to Select Features

In [72]:
# training and test dataset for Boruta
#X_train_n = X_train.drop( ['date', 'sales'], axis=1 ).values
#y_train_n = y_train.values.ravel()

# define RandomForestRegressor
#rf = RandomForestRegressor( n_jobs=-1 )

# define Boruta
#boruta = BorutaPy( rf, n_estimators='auto', verbose=2, random_state=42 ).fit( X_train_n, y_train_n )

### 6.2.1 Best Features from Boruta

In [73]:
#cols_selected = boruta.support_.tolist()

# best features
#X_train_fs = X_train.drop( ['date', 'sales'], axis=1 )
#cols_selected_boruta = X_train_fs.iloc[:, cols_selected].columns.to_list()

# not selected boruta
#cols_not_selected_boruta = list( np.setdiff1d( X_train_fs.columns, cols_selected_boruta ) )

## 6.3 Manual Feature Selection

In [74]:
df6.head()

In [75]:
cols_selected_boruta = [
    'store',
    'promo',
    'store_type',
    'assortment',
    'competition_distance',
    'competition_open_since_month',
    'competition_open_since_year',
    'promo2',
    'promo2_since_week',
    'promo2_since_year',
    'competition_time_month',
    'promo_time_week',
    'day_of_week_sin',
    'day_of_week_cos',
    'month_sin',
    'month_cos',
    'day_sin',
    'day_cos',
    'week_of_year_sin',
    'week_of_year_cos']

# columns to add
feat_to_add = ['date', 'sales']

cols_selected_boruta_full = cols_selected_boruta.copy()
cols_selected_boruta_full.extend( feat_to_add )

# 7.0 Model Data

In [76]:
x_train = X_train[cols_selected_boruta]
x_test = X_test[cols_selected_boruta]

# Time Series Data Preparation
x_training = X_train[cols_selected_boruta_full]

## 7.1 Average Model - Baseline

In [77]:
aux1 = x_test.copy()
aux1['sales'] = y_test.copy()

# prediction
aux2 = aux1[['store', 'sales']].groupby('store').mean().reset_index().rename( columns={'sales': 'predictions'} )
aux1 = pd.merge(aux1, aux2, how='left', on='store')
yhat_baseline = aux1['predictions']

# performance
baseline_result = ml_error('Average Model', np.expm1( y_test ), np.expm1( yhat_baseline ))
baseline_result

## 7.2 Linear Regression Model

In [78]:
 # model
lr = LinearRegression().fit(x_train, y_train)

# prediction
yhat_lr = lr.predict(x_test)

# performance
lr_result = ml_error('Linear Regression', np.expm1( y_test ), np.expm1( yhat_lr ))
lr_result

### 7.2.1 Linear Regression Model - Cross Validation

In [79]:
lr_result_cv = ts_cross_validation (x_training, 5, "Linear Regression", lr, verbose= False)
lr_result_cv

## 7.3 Linear Regression Regularized Model - Lasso

In [80]:
 # model
lrr = Lasso(alpha=0.01).fit(x_train, y_train)

# prediction
yhat_lrr = lrr.predict( x_test )

# performance
lrr_result = ml_error( 'Linear Regression - Lasso', np.expm1( y_test ), np.expm1( yhat_lrr ) )
lrr_result

### 7.3.1 Lasso - Cross Validation

In [81]:
lrr_result_cv = ts_cross_validation(x_training, 5, "Lasso", lrr, verbose=False)
lrr_result_cv

## 7.4  Random Forest Regressor

In [82]:
 # model
rf = RandomForestRegressor(n_estimators=100, random_state=42).fit( x_train, y_train)

# prediction
yhat_rf = rf.predict(x_test)

# performance
rf_result = ml_error('Random Forest Regressor', np.expm1(y_test), np.expm1(yhat_rf))
rf_result

### 7.4.1 Random Forest Regressor - Cross Validation

In [83]:
rf_result_cv = ts_cross_validation(x_training, 5, "Random Forest Regressor", rf, verbose=True)
rf_result_cv

## 7.5  XGBoost Regressor

In [84]:
 # model
model_xgb = xgb.XGBRegressor(objective='reg:squarederror',
                              n_estimators=100, 
                              eta=0.01, 
                              max_depth=10, 
                              subsample=0.7,
                              colsample_bytee=0.9).fit(x_train, y_train)

# prediction
yhat_xgb = model_xgb.predict(x_test)

# performance
xgb_result = ml_error('XGBoost Regressor', np.expm1(y_test), np.expm1(yhat_xgb ))
xgb_result

### 7.5.1 XGBoost Regressor - Cross Validation

In [85]:
xgb_result_cv = ts_cross_validation(x_training, 5, 'XGBoost Regressor', model_xgb, verbose=True)
xgb_result_cv

## 7.6 Comparing Models Performance

### 7.6.1 Single Performance

In [86]:
modelling_result = pd.concat([baseline_result, lr_result, lrr_result, rf_result, xgb_result])
modelling_result.sort_values('RMSE')

### 7.6.2 Real Performance - Cross Validation

In [87]:
modelling_result_cv = pd.concat([lr_result_cv, lrr_result_cv, rf_result_cv, xgb_result_cv])
modelling_result_cv

In [94]:
with open("model_result_cv.pkl", "wb") as f:
    pickle.dump(modelling_result_cv, f)

# 8.0 Hyperparameter fine tuning

## 8.1 Random Search

In [ ]:
# Random Search
# Grid Search

param = {'n_estimators': [int(x) for x in np.linspace(start = 1500, stop = 3500, num = 5)],
        'eta': [0.01, 0.03],
        'max_depth': [3, 5, 9],,
         'subsample': [0.1, 0.5, 0.7],
         'colsample_bytee': [0.3, 0.7, 0.9],
         'min_child_weight': [3, 8, 15],
        }

MAX_EVAL = 5

In [ ]:
#final result
final_result = pd.DataFrame()

for i in range ( MAX_EVAL ):
    hp = {k: random.sample (v, 1)[0] for k, v in param.items()}
    print(hp)
    
    model_xgb = xgb.XGBRegressor(objective='reg:squarederror',
                                n_estimators = hp['n_estimators'],
                                 eta = hp['eta'], 
                                 max_depth = hp['max_depth'],
                                 subsample = hp['subsample'],
                                 colsample_bytee = hp['colsample_bytee'],
                                 min_child_weight = hp['min_child_weight'])
    
    #performance
    
    result = cross_validation(x_training, 5, 'XGBoost Regressor', model_xgb, verbose=True)
    final_result = pd.concat([final_result, result])

In [ ]:
final_result

# 8.2 Final Model

In [103]:
param_tuned = {
    'n_estimators': 3000,
    'eta': 0.03,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytee': 0.7,
    'min_child_weight': 3
}

In [105]:
model_xgb_tuned = xgb.XGBRegressor(objective='reg:squarederror',
                                n_estimators = param_tuned['n_estimators'],
                                 eta = param_tuned['eta'], 
                                 max_depth = param_tuned['max_depth'],
                                 subsample = param_tuned['subsample'],
                                 colsample_bytee = param_tuned['colsample_bytee'],
                                 min_child_weight = param_tuned['min_child_weight']).fit(x_train, y_train)

#prediction
yhat_xgb_tuned = model_xgb_tuned.predict(x_test)


#performance
xgb_result_tuned = ml_error('XGBoost Regressor', np.expm1(y_test), np.expm1(yhat_xgb_tuned))
xgb_result_tuned


with open("model_rossmann.pkl", "wb") as f:
    pickle.dump(model_xgb_tuned, f)

In [106]:
mpe = mean_percentage_error(np.expm1(y_test), np.expm1(yhat_xgb_tuned))
mpe

# 9.0  Interpreting Results

In [112]:
df9 = X_test[cols_selected_boruta_full]

#rescale
df9["sales"] = np.expm1(df9["sales"])
df9["predictions"] = np.expm1(yhat_xgb_tuned)

In [113]:
df9.head()

## 9.1 Business Performance

In [114]:
# sum of predictions
df91 = df9[["store", "predictions"]].groupby("store").sum().reset_index()

#MAE and MAPE
df9_aux1 = df9[['store', 'sales', 'predictions']].groupby( 'store' ).apply( lambda x: mean_absolute_error( x['sales'], x['predictions'] ) ).reset_index().rename( columns={0:'MAE'})
df9_aux2 = df9[['store', 'sales', 'predictions']].groupby( 'store' ).apply( lambda x: mean_absolute_percentage_error( x['sales'], x['predictions'] ) ).reset_index().rename( columns={0:'MAPE'})

# Merge
df9_aux3 = pd.merge( df9_aux1, df9_aux2, how='inner', on='store' )
df92 = pd.merge( df91, df9_aux3, how='inner', on='store' )

# Scenarios
df92['worst_scenario'] = df92['predictions'] - df92['MAE']
df92['best_scenario'] = df92['predictions'] + df92['MAE']

# order columns
df92 = df92[['store', 'predictions', 'worst_scenario', 'best_scenario', 'MAE', 'MAPE']]

In [115]:
df92.sort_values("MAPE", ascending=False).head()

In [116]:
sns.scatterplot(x = "store", y = "MAPE", data = df92)

## 9.2 Total Performance

In [118]:
df93 = df92[["predictions", "worst_scenario", "best_scenario"]].apply(lambda x: np.sum(x), axis = 0).reset_index().rename(columns = {"index" : "Scenario", 0: "Values"})
df93["Values"] = df93["Values"].map("R${:,.2f}".format)
df93

## 9.3 Machine Learning Performance

In [119]:
df9['error'] = df9['sales'] - df9['predictions']
df9['error_rate'] = df9['predictions'] / df9['sales']

In [120]:
plt.subplot( 2, 2, 1 )
sns.lineplot( x='date', y='sales', data=df9, label='SALES' )
sns.lineplot( x='date', y='predictions', data=df9, label='PREDICTIONS' )

plt.subplot( 2, 2, 2 )
sns.lineplot( x='date', y='error_rate', data=df9 )
plt.axhline( 1, linestyle='--')

plt.subplot( 2, 2, 3 )
sns.distplot( df9['error'] )

plt.subplot( 2, 2, 4 )
sns.scatterplot( df9['predictions'], df9['error'] )